In [1]:
import warnings
warnings.filterwarnings("ignore")
import pandas as pd
import matplotlib.pyplot as plt
import torchvision as T
import torch
from sklearn import preprocessing
import numpy as np
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from torch.utils.data import Dataset, DataLoader
import math
import itertools
import os
import sys
root_path = os.path.abspath(os.path.join('../..')) # <- adjust such that root_path always points at the root project dir (i.e. if current file is two folders deep, use '../..'). 
if root_path not in sys.path:
    sys.path.append(root_path)
import database_server.db_utilities as dbu 
import pickle as pkl
from sklearn.decomposition import PCA
from LSTM_help_functions import preprocess, game_dict, inputs, inputs_2seas, club_dict, points_and_co, points_and_co_oppon, data_to_lstm, Sport_pred_2LSTM_1, two_team_inputs
import random
from LSTM_prediction import lstm_setup, sequence_models, sequence_models_2gru, sequence_models_goalpred
from gru_models import Sport_pred_1GRU_3, Sport_pred_2GRU_1

In [2]:
torch.manual_seed(69420)
random.seed(69420)
np.random.seed(69420)

In [3]:
import pickle
with open('clubs_gru.pkl', 'rb') as f:
    clubs = pickle.load(f)

In [4]:
with open('rearrange_list_gru.pkl', 'rb') as f:
    rearrange_list = pickle.load(f)

In [5]:
with open('scale_df_gru.pkl', 'rb') as f:
    scale_df = pickle.load(f)

In [6]:
with open('result_dict_gru.pkl', 'rb') as f:
    result_dict = pickle.load(f)

In [7]:
with open('venue_dict_gru.pkl', 'rb') as f:
    venue_dict = pickle.load(f)

# 1GRU outcome prediction 

In [25]:
net = torch.load(os.path.join(root_path, "models", "neural_net", "saved_models", "1GRU_3_outcome.pth"))
net.eval()

Sport_pred_1GRU_3(
  (l_lstm1): GRU(369, 369, num_layers=2, batch_first=True, dropout=0.2)
  (l_linear1): Linear(in_features=369, out_features=3, bias=True)
  (soft): Softmax(dim=None)
  (relu): ReLU()
  (sigm): Sigmoid()
)

In [9]:
sequence_models(net, 88, 65, clubs, rearrange_list, scale_df, result_dict, venue_dict)

,home_win_prob,draw_prob,away_win_prob
0,0.331521,0.280523,0.387956


# 2GRU outcome prediction

In [24]:
net2 = torch.load(os.path.join(root_path, "models", "neural_net", "saved_models", "2GRU_1_outcome.pth"))
net2.eval()

Sport_pred_2GRU_1(
  (l_lstm1): GRU(369, 369, num_layers=2, batch_first=True, dropout=0.2)
  (l_lstm2): GRU(369, 369, num_layers=2, batch_first=True, dropout=0.2)
  (l_linear1): Linear(in_features=738, out_features=738, bias=True)
  (l_linear2): Linear(in_features=738, out_features=3, bias=True)
  (soft): Softmax(dim=None)
  (relu): ReLU()
  (sigm): Sigmoid()
)

In [11]:
sequence_models_2gru(net2, 88, 65, clubs, rearrange_list, scale_df, result_dict, venue_dict)

,home_win_prob,draw_prob,away_win_prob
0,0.283996,0.303984,0.41202


# 1GRU goal prediction

In [26]:
net3 = torch.load(os.path.join(root_path, "models", "neural_net", "saved_models", "1GRU_3_goals.pth"))
net3.eval()

Sport_pred_1GRU_3(
  (l_lstm1): GRU(370, 370, num_layers=2, batch_first=True, dropout=0.2)
  (l_linear1): Linear(in_features=370, out_features=10, bias=True)
  (soft): Softmax(dim=None)
  (relu): ReLU()
  (sigm): Sigmoid()
)

In [13]:
with open('rearrange_list_gru_goals.pkl', 'rb') as f:
    rearrange_list_goals = pickle.load(f)

In [19]:
home, away = sequence_models_goalpred(net3, 11, 65, clubs, rearrange_list_goals, scale_df, result_dict, venue_dict)

In [20]:
home

,0,1,2,3,4,5,6,7,8,9
0,0.320536,0.35542,0.184565,0.102025,0.024786,0.008939,0.002165,0.000762,0.000367,0.000434


In [21]:
away

,0,1,2,3,4,5,6,7,8,9
0,0.369541,0.350464,0.185828,0.067833,0.018705,0.005598,0.001407,0.00032,0.000149,0.000156


In [27]:
os.getcwd()

'/Users/paul/Documents/Uni/SoSe23/dsp2/DS-Project/models/neural_net'